In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

In [2]:
# separate columns by ;
df = pd.read_csv('../../datasets/atis_dataset_converted_update_2.csv', sep=';')

In [3]:
df.head()

,atis_id,airport_icao,visibility,wind_speed,wind_gust,wind_direction,rvr,runway_designator_number,runway_designator_side,runway_ils_category,headwind,crosswind,ceiling,weather_phenomenon,ILS,RNAV,RNP,VISUAL
0,1,KIAD,10.0,6,0,180,>6000,19,C,No Category,5.9,-1.0,99999,NaN,1,0,0,1
1,1,KIAD,10.0,6,0,180,>6000,19,L,No Category,5.9,-1.0,99999,NaN,1,0,0,1
2,1,KIAD,10.0,6,0,180,>6000,19,R,No Category,5.9,-1.0,99999,NaN,1,0,0,1
3,2,KLAX,10.0,4,0,40,0Â,24,R,No ILS,-3.8,1.4,99999,NaN,0,0,1,0
4,3,KSFO,10.0,8,0,260,>6000,28,L,No Category,7.5,-2.7,99999,NaN,1,0,0,0


In [4]:
df.iloc[22]

atis_id                              16
airport_icao                       KORD
visibility                         10.0
wind_speed                            4
wind_gust                             0
wind_direction                      240
rvr                               >6000
runway_designator_number             27
runway_designator_side                R
runway_ils_category         No Category
headwind                            3.5
crosswind                          -2.0
ceiling                           99999
weather_phenomenon                  NaN
ILS                                   1
RNAV                                  0
RNP                                   0
VISUAL                                0
Name: 22, dtype: object

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21077 entries, 0 to 21076
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   atis_id                   21077 non-null  int64  
 1   airport_icao              21077 non-null  object 
 2   visibility                21074 non-null  float64
 3   wind_speed                21077 non-null  int64  
 4   wind_gust                 21077 non-null  int64  
 5   wind_direction            21077 non-null  int64  
 6   rvr                       21077 non-null  object 
 7   runway_designator_number  21077 non-null  int64  
 8   runway_designator_side    20862 non-null  object 
 9   runway_ils_category       21077 non-null  object 
 10  headwind                  21077 non-null  float64
 11  crosswind                 21077 non-null  float64
 12  ceiling                   21077 non-null  int64  
 13  weather_phenomenon        1689 non-null   object 
 14  ILS   

In [6]:
# check null values before preprocess
df.isnull().sum()

atis_id                         0
airport_icao                    0
visibility                      3
wind_speed                      0
wind_gust                       0
wind_direction                  0
rvr                             0
runway_designator_number        0
runway_designator_side        215
runway_ils_category             0
headwind                        0
crosswind                       0
ceiling                         0
weather_phenomenon          19388
ILS                             0
RNAV                            0
RNP                             0
VISUAL                          0
dtype: int64

In [7]:
# delete rows where rvr = FFF and rvr = 0
df = df[df['rvr'] != 'FFF']
df = df[df['rvr'] != '0Â']

In [8]:
# change N/A weather_phenomenon to "No Phenomenon"
df['weather_phenomenon'] = df['weather_phenomenon'].fillna('No Phenomenon')

In [9]:
# fill empty runway_designator_side to X
df['runway_designator_side'] = df['runway_designator_side'].fillna('X') # X means no L, R, or C

In [10]:
# clear whitespaces in rvr column
df['rvr'] = df['rvr'].str.strip()

In [11]:
# extract non-digit characters only (excluding decimal point)
df['rvr_tendency'] = df['rvr'].astype(str).apply(lambda x: ''.join(re.findall(r'\D+', x)).strip())
df['rvr'] = df['rvr'].astype(str).str.extract(r'(\d+)', expand=False).astype(float)

In [12]:
# change rvr_tendency empty string ('Â') to 'Stable'
df.loc[df.rvr_tendency == 'Â', 'rvr_tendency'] = 'Stable'

# change rvr_tendency '>' symbol to 'Excellent'
df.loc[df.rvr_tendency == '>', 'rvr_tendency'] = 'Excellent'

# change rvr_tendency '▲/â–²' symbol to 'Improving'
df.loc[df.rvr_tendency == 'â–²', 'rvr_tendency'] = 'Improving'

# change rvr_tendency '▼/â–¼' symbol to 'Deteriorating'
df.loc[df.rvr_tendency == 'â–¼', 'rvr_tendency'] = 'Deteriorating'

In [13]:
# remove the column temporarily
rvr_tendency = df.pop('rvr_tendency')

# find the index of 'rvr' column
rvr_index = df.columns.get_loc('rvr')

# insert it right after 'rvr'
df.insert(rvr_index + 1, 'rvr_tendency', rvr_tendency)


In [14]:
# drop atis_id column
df.drop(columns=['atis_id'], inplace=True)

In [15]:
# update KIAD runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KIAD)
# Step 1: Set all ILS runways at KIAD to CAT I by default
kiad_condition = (df['airport_icao'] == 'KIAD')
df.loc[kiad_condition, 'runway_ils_category'] = 'CAT I'

# Step 2: Explicit ILS capability mapping
ils_mapping_kiad = {
    ('KIAD', 1, 'C'): 'CAT II',
    ('KIAD', 1, 'L'): 'CAT III',
    ('KIAD', 1, 'R'): 'CAT III',
    ('KIAD', 19, 'C'): 'CAT III',
    ('KIAD', 19, 'L'): 'CAT II',
    ('KIAD', 19, 'R'): 'CAT III',
}

# Step 3: Apply mapping
def update_ils_category_kiad(row):
    key = (row['airport_icao'], row['runway_designator_number'], row['runway_designator_side'])
    return ils_mapping_kiad.get(key, row['runway_ils_category'])

df['runway_ils_category'] = df.apply(update_ils_category_kiad, axis=1)

df.loc[
    (df['airport_icao'] == 'KIAD') & (df['runway_designator_number'] == 19) & (df['runway_designator_side'] == 'L'), 'runway_ils_category'
] = 'CAT II'

In [16]:
# update KLAX runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KLAX)
# Set all KLAX runways with ILS/LOC to CAT I by default
df.loc[(df['airport_icao'] == 'KLAX'), 'runway_ils_category'] = 'CAT I'

# Then upgrade those with CAT III capability
df.loc[(df['airport_icao'] == 'KLAX') & (df['runway_designator_number'].isin([24, 25])) &
    (df['runway_designator_side'] == 'R') | (df['runway_designator_side'] == 'L'),
    'runway_ils_category'] = 'CAT III'

In [17]:
# update KATL runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KATL)
# Step 1: Default all ILS runways at KATL to CAT I
katl_condition = (df['airport_icao'] == 'KATL')
df.loc[katl_condition, 'runway_ils_category'] = 'CAT I'

# Step 2: Override based on max available CAT
ils_mapping_katl = {
    ('KATL', 8, 'L'): 'CAT III',
    ('KATL', 9, 'R'): 'CAT III',
    ('KATL', 10, 'X'): 'CAT III',
    ('KATL', 26, 'R'): 'CAT II',
    ('KATL', 27, 'L'): 'CAT II',
    ('KATL', 28, 'X'): 'CAT II',
}

def update_ils_category_katl(row):
    key = (row['airport_icao'], row['runway_designator_number'], row['runway_designator_side'])
    return ils_mapping_katl.get(key, row['runway_ils_category'])

df['runway_ils_category'] = df.apply(update_ils_category_katl, axis=1)

In [18]:
# update KSFO runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KSFO)
# Step 1: Default all KSFO ILS runways to CAT I
ksfo_condition = (df['airport_icao'] == 'KSFO')
df.loc[ksfo_condition, 'runway_ils_category'] = 'CAT I'

# Step 2: Manual override for better-equipped runways
ils_mapping_ksfo = {
    ('KSFO', 28, 'L'): 'CAT II',
    ('KSFO', 28, 'R'): 'CAT III',
}

# Step 3: Apply the mapping
def update_ils_category_ksfo(row):
    key = (row['airport_icao'], row['runway_designator_number'], row['runway_designator_side'])
    return ils_mapping_ksfo.get(key, row['runway_ils_category'])

df['runway_ils_category'] = df.apply(update_ils_category_ksfo, axis=1)

In [19]:
# update KORD runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KORD)
# Step 1: Default all KORD ILS runways to CAT I
kord_condition = (df['airport_icao'] == 'KORD')
df.loc[kord_condition, 'runway_ils_category'] = 'CAT I'

# Step 2: Override based on specific mappings
ils_mapping_kord = {
    ('KORD', 4, 'R'): 'CAT II',
    ('KORD', 9, 'C'): 'CAT III',
    ('KORD', 9, 'L'): 'CAT III',
    ('KORD', 9, 'R'): 'CAT III',
    ('KORD', 10, 'C'): 'CAT III',
    ('KORD', 10, 'L'): 'CAT III',
    ('KORD', 10, 'R'): 'CAT III',
    ('KORD', 22, 'L'): 'CAT II',
    ('KORD', 22, 'R'): 'CAT I',
    ('KORD', 27, 'C'): 'CAT III',
    ('KORD', 27, 'L'): 'CAT III',
    ('KORD', 27, 'R'): 'CAT III',
    ('KORD', 28, 'C'): 'CAT III',
    ('KORD', 28, 'L'): 'CAT III',
    ('KORD', 28, 'R'): 'CAT III',
}

def update_ils_category_kord(row):
    key = (row['airport_icao'], row['runway_designator_number'], row['runway_designator_side'])
    return ils_mapping_kord.get(key, row['runway_ils_category'])

df['runway_ils_category'] = df.apply(update_ils_category_kord, axis=1)

In [20]:
# update KJFK runway_ils_category to match real world ILS category for each runway
# according to (http://www.airnav.com/airport/KJFK)
# Step 1: Default all ILS runways at KJFK to CAT I
kjfk_condition = (df['airport_icao'] == 'KJFK')
df.loc[kjfk_condition, 'runway_ils_category'] = 'CAT I'

# Step 2: Specific overrides based on actual CAT info
ils_mapping_kjfk = {
    ('KJFK', 13, 'L'): 'CAT II',
    ('KJFK', 4, 'R'): 'CAT III',
    ('KJFK', 22, 'L'): 'CAT III',
}

# Step 3: Apply mapping
def update_ils_category_kjfk(row):
    key = (row['airport_icao'], row['runway_designator_number'], row['runway_designator_side'])
    return ils_mapping_kjfk.get(key, row['runway_ils_category'])

df['runway_ils_category'] = df.apply(update_ils_category_kjfk, axis=1)

In [21]:
# check after preprocessed
df.head()

,airport_icao,visibility,wind_speed,wind_gust,wind_direction,rvr,rvr_tendency,runway_designator_number,runway_designator_side,runway_ils_category,headwind,crosswind,ceiling,weather_phenomenon,ILS,RNAV,RNP,VISUAL
0,KIAD,10.0,6,0,180,6000.0,Excellent,19,C,CAT III,5.9,-1.0,99999,No Phenomenon,1,0,0,1
1,KIAD,10.0,6,0,180,6000.0,Excellent,19,L,CAT III,5.9,-1.0,99999,No Phenomenon,1,0,0,1
2,KIAD,10.0,6,0,180,6000.0,Excellent,19,R,CAT III,5.9,-1.0,99999,No Phenomenon,1,0,0,1
4,KSFO,10.0,8,0,260,6000.0,Excellent,28,L,CAT II,7.5,-2.7,99999,No Phenomenon,1,0,0,0
5,KATL,10.0,0,0,0,6000.0,Excellent,27,L,CAT II,0.0,0.0,99999,No Phenomenon,0,0,0,1


In [22]:
df.iloc[22]

airport_icao                         KJFK
visibility                           10.0
wind_speed                              7
wind_gust                               0
wind_direction                        190
rvr                                6000.0
rvr_tendency                    Excellent
runway_designator_number               22
runway_designator_side                  R
runway_ils_category                 CAT I
headwind                              6.1
crosswind                            -3.5
ceiling                             24000
weather_phenomenon          No Phenomenon
ILS                                     1
RNAV                                    0
RNP                                     0
VISUAL                                  0
Name: 25, dtype: object

In [23]:
# check null values after preprocessed
df.isnull().sum()

airport_icao                0
visibility                  3
wind_speed                  0
wind_gust                   0
wind_direction              0
rvr                         0
rvr_tendency                0
runway_designator_number    0
runway_designator_side      0
runway_ils_category         0
headwind                    0
crosswind                   0
ceiling                     0
weather_phenomenon          0
ILS                         0
RNAV                        0
RNP                         0
VISUAL                      0
dtype: int64

In [24]:
# see rows with null value in any column
df[df.isnull().any(axis=1)]

,airport_icao,visibility,wind_speed,wind_gust,wind_direction,rvr,rvr_tendency,runway_designator_number,runway_designator_side,runway_ils_category,headwind,crosswind,ceiling,weather_phenomenon,ILS,RNAV,RNP,VISUAL
15542,KIAD,NaN,9,0,170,6000.0,Excellent,30,X,CAT I,-5.8,-6.9,99999,No Phenomenon,0,0,0,0
15553,KIAD,NaN,8,0,170,6000.0,Excellent,30,X,CAT I,-5.1,-6.1,99999,No Phenomenon,0,0,0,0
17750,KATL,NaN,23,36,310,6000.0,Excellent,26,R,CAT II,14.8,17.6,99999,No Phenomenon,1,0,0,0


In [25]:
# delete rows that have null value(s)
df.dropna(inplace=True)

In [26]:
# show nan values in X
df.isnull().sum()

airport_icao                0
visibility                  0
wind_speed                  0
wind_gust                   0
wind_direction              0
rvr                         0
rvr_tendency                0
runway_designator_number    0
runway_designator_side      0
runway_ils_category         0
headwind                    0
crosswind                   0
ceiling                     0
weather_phenomenon          0
ILS                         0
RNAV                        0
RNP                         0
VISUAL                      0
dtype: int64

In [27]:
# see columns datatypes after preprocessed
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20815 entries, 0 to 21076
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   airport_icao              20815 non-null  object 
 1   visibility                20815 non-null  float64
 2   wind_speed                20815 non-null  int64  
 3   wind_gust                 20815 non-null  int64  
 4   wind_direction            20815 non-null  int64  
 5   rvr                       20815 non-null  float64
 6   rvr_tendency              20815 non-null  object 
 7   runway_designator_number  20815 non-null  int64  
 8   runway_designator_side    20815 non-null  object 
 9   runway_ils_category       20815 non-null  object 
 10  headwind                  20815 non-null  float64
 11  crosswind                 20815 non-null  float64
 12  ceiling                   20815 non-null  int64  
 13  weather_phenomenon        20815 non-null  object 
 14  ILS        

In [28]:
df.describe()

,visibility,wind_speed,wind_gust,wind_direction,rvr,runway_designator_number,headwind,crosswind,ceiling,ILS,RNAV,RNP,VISUAL
count,20815.000000,20815.000000,20815.000000,20815.00000,20815.000000,20815.000000,20815.000000,20815.000000,20815.000000,20815.000000,20815.00000,20815.000000,20815.000000
mean,9.427498,9.147922,5.100312,189.54749,5973.043478,19.842710,6.181696,-1.216675,47774.150709,0.646841,0.04958,0.196733,0.616430
std,1.810697,5.354869,10.294945,103.89079,303.400279,9.343871,6.044216,6.015483,44622.764591,0.477963,0.21708,0.397538,0.486267
min,0.062500,0.000000,0.000000,-1.00000,0.000000,1.000000,-13.800000,-25.600000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,10.000000,5.000000,0.000000,110.00000,6000.000000,10.000000,2.000000,-4.650000,5000.000000,0.000000,0.00000,0.000000,0.000000
50%,10.000000,8.000000,0.000000,210.00000,6000.000000,24.000000,5.900000,-0.900000,25000.000000,1.000000,0.00000,0.000000,1.000000
75%,10.000000,12.000000,0.000000,260.00000,6000.000000,27.000000,10.000000,2.300000,99999.000000,1.000000,0.00000,0.000000,1.000000
max,10.000000,32.000000,43.000000,360.00000,6000.000000,31.000000,32.000000,25.600000,99999.000000,1.000000,1.00000,1.000000,1.000000


In [29]:
# Save the cleaned DataFrame
df.to_csv('../../datasets/atis_dataset_preprocessed.csv', sep=';', index=False)
df.to_csv('atis_dataset_preprocessed.csv', sep=';', index=False)